In [7]:
import numpy as np
import pandas as pd
from sklearn.metrics import mean_absolute_error as mae, mean_squared_error as mse, r2_score as r2
from sklearn.preprocessing import StandardScaler
from sklearn.neural_network import MLPRegressor
from sklearn.datasets import fetch_california_housing
import model
from model import read_data_model, add_laplace_noise

np.random.seed(42)

# Cargar los datos
all_data = fetch_california_housing()
X_train_val, X_val, X_test, Y_train_val, Y_val, Y_test = read_data_model(all_data, 'MedInc')

# Escalado de los datos
scaler = StandardScaler()
X_train_val_scaled = scaler.fit_transform(X_train_val)
X_val_scaled = scaler.transform(X_val)
X_test_scaled = scaler.transform(X_test)

# Definir los parámetros para MLPRegressor
mlp_params = {
    'hidden_layer_sizes': (50, 50),
    'activation': 'relu',
    'alpha': 0.0001,
    'learning_rate': 'constant',
    'solver': 'adam',
    'max_iter': 500,
    'random_state': 42
}

# Crear el modelo de red neuronal MLPRegressor con los parámetros especificados
model_original = MLPRegressor(**mlp_params)
model_original.fit(X_train_val_scaled, Y_train_val)
Y_predict = model_original.predict(X_test_scaled)

# Evaluar el modelo original (sin ruido)
MSE_original = mse(Y_test, Y_predict)
MAE_original = mae(Y_test, Y_predict)
RMSE_original = np.sqrt(MSE_original)
R2_original = r2(Y_test, Y_predict)

print("\nPredicción de precios de casas sin ruido:")
print(f"MSE: {MSE_original}")
print(f"MAE: {MAE_original}")
print(f"RMSE: {RMSE_original}")
print(f"R2: {R2_original}")

# Paso 2: Crear nuevos datos de entrada reemplazando solo la columna de ingreso con las predicciones
column_index = list(all_data.feature_names).index('MedInc')  # Índice de la columna de ingreso
X_train_val_new = np.copy(X_train_val)
X_test_new = np.copy(X_test)

# Realiza la predicción en el conjunto de entrenamiento para el modelo original
Y_predict_train = model_original.predict(X_train_val_scaled)

# Reemplazar solo la columna de ingreso
Y_train_val_new = X_train_val_new[:, column_index].copy()
Y_test_new = X_test_new[:, column_index].copy()
X_train_val_new[:, column_index] = Y_predict_train  
X_test_new[:, column_index] = Y_predict 

scaler_reconstruct = StandardScaler()
X_train_val_new_scaled = scaler_reconstruct.fit_transform(X_train_val_new)
X_test_new_scaled = scaler_reconstruct.transform(X_test_new)

# Reconstrucción de la columna de ingreso usando los datos sin ruido
model_reconstruct = MLPRegressor(**mlp_params)
model_reconstruct.fit(X_train_val_new_scaled, Y_train_val_new)
income_predict = model_reconstruct.predict(X_test_new_scaled)

# Evaluar la reconstrucción de ingresos (sin ruido)
MSE_income_reconstruct = mse(X_test[:, column_index], income_predict)
MAE_income_reconstruct = mae(X_test[:, column_index], income_predict)
RMSE_income_reconstruct = np.sqrt(MSE_income_reconstruct)
R2_income_reconstruct = r2(X_test[:, column_index], income_predict)

print("\nReconstrucción de ingresos (sin ruido):")
print(f"MSE: {MSE_income_reconstruct}")
print(f"MAE: {MAE_income_reconstruct}")
print(f"RMSE: {RMSE_income_reconstruct}")
print(f"R2: {R2_income_reconstruct}")

# Paso 4: Añadir ruido Laplaciano a la columna de ingreso y repetir el procedimiento
epsilon = 1 # Usamos el epsilon ya seleccionado
X_train_val_noisy = add_laplace_noise(X_train_val, epsilon)
scalerRuido = StandardScaler()
X_train_val_noisy_scaled = scalerRuido.fit_transform(X_train_val_noisy)
X_test_scaled = scalerRuido.transform(X_test)

# Entrenar el modelo con datos ruidosos
model_noisy = MLPRegressor(**mlp_params)
model_noisy.fit(X_train_val_noisy_scaled, Y_train_val)
Y_predict_noisy = model_noisy.predict(X_test_scaled)

# Evaluar el modelo con datos ruidosos
MSE_noisy = mse(Y_test, Y_predict_noisy)
MAE_noisy = mae(Y_test, Y_predict_noisy)
RMSE_noisy = np.sqrt(MSE_noisy)
R2_noisy = r2(Y_test, Y_predict_noisy)

print("\nPredicción de precios con ruido:")
print(f"MSE: {MSE_noisy}")
print(f"MAE: {MAE_noisy}")
print(f"RMSE: {RMSE_noisy}")
print(f"R2: {R2_noisy}")

# Paso 5: Crear nuevos datos de entrada reemplazando la columna de ingreso con las predicciones ruidosas
X_train_val_new_noisy = np.copy(X_train_val_noisy)
X_test_new = np.copy(X_test)

# Realiza la predicción en el conjunto de entrenamiento con el modelo ruidoso
Y_predict_train_noisy = model_noisy.predict(X_train_val_noisy_scaled)

# Reemplazar solo la columna de ingreso con las predicciones ruidosas
Y_train_val_new = X_train_val_new_noisy[:, column_index].copy()
Y_test_new = X_test_new[:, column_index].copy()
X_train_val_new_noisy[:, column_index] = Y_predict_train_noisy  
X_test_new[:, column_index] = Y_predict

scaler_reconstruct = StandardScaler()
X_train_val_new_scaled_noisy = scaler_reconstruct.fit_transform(X_train_val_new_noisy)
X_test_new_scaled = scaler_reconstruct.transform(X_test_new)

# Reconstrucción de la columna de ingreso con datos ruidosos
model_reconstruct_noisy = MLPRegressor(**mlp_params)
model_reconstruct_noisy.fit(X_train_val_new_scaled_noisy, Y_train_val_new)
income_predict_noisy = model_reconstruct_noisy.predict(X_test_new_scaled)

# Evaluar la reconstrucción de ingresos (con ruido)
MSE_income_reconstruct_noisy = mse(X_test[:, column_index], income_predict_noisy)
MAE_income_reconstruct_noisy = mae(X_test[:, column_index], income_predict_noisy)
RMSE_income_reconstruct_noisy = np.sqrt(MSE_income_reconstruct_noisy)
R2_income_reconstruct_noisy = r2(X_test[:, column_index], income_predict_noisy)

print("\nReconstrucción de ingresos (con ruido):")
print(f"MSE: {MSE_income_reconstruct_noisy}")
print(f"MAE: {MAE_income_reconstruct_noisy}")
print(f"RMSE: {RMSE_income_reconstruct_noisy}")
print(f"R2: {R2_income_reconstruct_noisy}")



Predicción de precios de casas sin ruido:
MSE: 0.22675958692747997
MAE: 0.33242608815868235
RMSE: 0.47619280436340067
R2: 0.7613851972611834

Reconstrucción de ingresos (sin ruido):
MSE: 0.23693061078643085
MAE: 0.3623738675949441
RMSE: 0.4867551856800612
R2: 0.8992847014332563

Predicción de precios con ruido:
MSE: 0.2610520964097098
MAE: 0.36057040840097354
RMSE: 0.5109325752090091
R2: 0.7252998414162806

Reconstrucción de ingresos (con ruido):
MSE: 22.905381792456467
MAE: 3.6820606462834555
RMSE: 4.785956726972828
R2: -8.736700371282828
